Загружаем данные о графе

In [1]:
!wget https://snap.stanford.edu/data/wiki-Vote.txt.gz
!gzip -d ./wiki-Vote.txt.gz

--2024-04-19 21:04:09--  https://snap.stanford.edu/data/wiki-Vote.txt.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80, 198.41.0.4, 2001:503:ba3e::2:30, ...
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 290339 (284K) [application/x-gzip]
Saving to: ‘wiki-Vote.txt.gz’

wiki-Vote.txt.gz    100%[===================>] 283.53K   278KB/s    in 1.0s    

2024-04-19 21:04:11 (278 KB/s) - ‘wiki-Vote.txt.gz’ saved [290339/290339]

gzip: ./wiki-Vote.txt already exists; do you wish to overwrite (y or n)? ^C


In [2]:
!git clone https://github.com/OafishSpark/InfluencerSearchAlgorithms

!mv /content/InfluencerSearchAlgorithms/* /content/
!rm -rf /content/InfluencerSearchAlgorithms

Cloning into 'InfluencerSearchAlgorithms'...
remote: Enumerating objects: 96, done.
remote: Counting objects: 100% (96/96), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 96 (delta 40), reused 84 (delta 28), pack-reused 0
Receiving objects: 100% (96/96), 16.10 KiB | 1.46 MiB/s, done.
Resolving deltas: 100% (40/40), done.
mv: cannot stat '/content/InfluencerSearchAlgorithms/*': No such file or directory


In [1]:
import networkx as nx

import numpy as np
from numpy import random

from tqdm import tqdm

In [2]:
from models.threshold_model import linear_threshold_model
from models.sir_model import sir_model, sir_inf_model

from influencers.greedy_kkt import greedy_kkt_influencers
from influencers.centrality import centrality_influencers
from influencers.shapley_value import shapley_value_influencers
from influencers.csr import csr_influencers
from influencers.hierarchy import hierarchy_influencers_dull, hierarchy_influencers_with_csr, hierarchy_influencers_with_centrality

from communities.clustering import clustering_communities
from communities.shapley_value import shapley_value_communities

from communities.hierarchy import hierarchy_communities_original, hierarchy_communities_improved

In [3]:
graph_congress = nx.read_edgelist("./congress_network/congress.edgelist", create_using=nx.DiGraph())

In [4]:
graph_wiki_votes = nx.read_edgelist("./wiki-Vote.txt", create_using=nx.DiGraph())
nx.set_edge_attributes(graph_wiki_votes, 1, 'weight')

In [5]:
subgraph_wiki_votes = nx.subgraph(graph_wiki_votes, list(graph_wiki_votes.nodes)[:300])

In [6]:
graphs = {
  "graph congress": graph_congress,
  "graph wiki votes": subgraph_wiki_votes,
}

In [7]:
for name in graphs.keys():
  print(name)
  graph = graphs[name]
  print(graph.number_of_nodes())
  print(graph.number_of_edges())
  print(nx.density(graph))
  print(nx.number_strongly_connected_components(graph))
  print("")

graph congress
475
13289
0.05902287363979569
7

graph wiki votes
300
3640
0.04057971014492753
123



In [8]:
graph = graphs["graph congress"]

Communities

In [ ]:
communities = {
    "communities shapley": shapley_value_communities(graph, 10),
    "communities clusters": clustering_communities(graph, 10),
    "communities hierarchy original": hierarchy_communities_original(graph, 10),
    "communities hierarchy improved": hierarchy_communities_improved(graph, 10)
}

Influencers

In [ ]:
influencers_pack = {
    "influencers hierarchy with centrality": hierarchy_influencers_with_centrality(graph, 10),
    "influencers shapley": shapley_value_influencers(graph, 10),
    "influencers centrality": centrality_influencers(graph, 10),
    "influencers kkt": greedy_kkt_influencers(graph, 10),
    "influencers hierarchy orig": hierarchy_influencers_dull(graph, 10),
    "influencers hierarchy improved": hierarchy_influencers_with_csr(graph, 10),
}

In [ ]:
influencers_csr = dict(zip(
    ['csr ' + name for name in communities.keys()],
    [csr_influencers(graph, 10, community) for community in communities.values()]
))

In [50]:
influencers_pack.update(influencers_csr)

In [51]:
def perform_experiment(
    n_experiments: int,
    graph: nx.DiGraph,
    influencers_dict: dict,
    time: int = 5,
) -> dict:
  nodes = list(graph.nodes)
  n_nodes = len(nodes)
  results = dict(zip(
    ["linear threshold model", "SIR model", "Infinite SIR model"],
    [dict([(name, 0) for name in influencers_dict.keys()]) for _ in range(3)]
  ))
  for _ in tqdm(range(n_experiments)):
    theta = dict(zip(list(graph.nodes), random.uniform(0.01, 0.99, graph.number_of_nodes())))
    # infection_prob = dict(zip(nodes, random.uniform(0.01, 0.99, n_nodes)))
    # recover_prob = dict(zip(nodes, random.uniform(0.01, 0.99, n_nodes)))
    for name in influencers_dict.keys():
      influencers = influencers_dict[name]
      results['linear threshold model'][name] += len(linear_threshold_model(graph, influencers, time, theta)) / n_experiments
      results['SIR model'][name] += len(sir_model(graph, influencers, time)) / n_experiments
  for name in influencers_dict.keys():
    influencers = influencers_dict[name]
    results['Infinite SIR model'][name] += len(sir_inf_model(graph, influencers, time))
  return results

In [63]:
number_of_experiments = 10

In [65]:
times = [2*i + 1 for i in range(10)]
results = dict(zip(
    times,
    [perform_experiment(number_of_experiments, graph, influencers_pack, time) for time in times]
))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:34<00:00,  3.44s/it]


In [66]:
times

[1, 3, 5, 7, 9, 11, 13, 15, 17, 19]

In [67]:
results

{1: {'linear threshold model': {'influencers hierarchy with centrality': 23.599999999999998,
   'influencers shapley': 28.599999999999998,
   'influencers centrality': 57.00000000000001,
   'influencers kkt': 27.200000000000003,
   'influencers hierarchy orig': 17.900000000000002,
   'influencers hierarchy improved': 23.599999999999998,
   'csr communities shapley': 28.8,
   'csr communities clusters': 16.700000000000003,
   'csr communities hierarchy original': 13.600000000000001,
   'csr communities hierarchy improved': 24.0},
  'SIR model': {'influencers hierarchy with centrality': 163.9,
   'influencers shapley': 110.30000000000001,
   'influencers centrality': 303.8,
   'influencers kkt': 125.20000000000002,
   'influencers hierarchy orig': 79.4,
   'influencers hierarchy improved': 163.2,
   'csr communities shapley': 177.60000000000002,
   'csr communities clusters': 111.79999999999998,
   'csr communities hierarchy original': 33.5,
   'csr communities hierarchy improved': 130.7

In [70]:
for results_t in results.values():
    for model in results_t.keys():
      print(model + ":")
      for infl_alg in sorted(results_t[model], reverse=True, key=lambda x: results_t[model][x])[:5]:
        print(infl_alg + ": " + str(results_t[model][infl_alg]))
      print('\n')

linear threshold model:
influencers centrality: 57.00000000000001
csr communities shapley: 28.8
influencers shapley: 28.599999999999998
influencers kkt: 27.200000000000003
csr communities hierarchy improved: 24.0


SIR model:
influencers centrality: 303.8
csr communities shapley: 177.60000000000002
influencers hierarchy with centrality: 163.9
influencers hierarchy improved: 163.2
csr communities hierarchy improved: 130.7


Infinite SIR model:
influencers centrality: 380
influencers hierarchy with centrality: 269
influencers hierarchy improved: 269
csr communities shapley: 264
influencers kkt: 201


linear threshold model:
influencers centrality: 131.0
influencers hierarchy with centrality: 56.4
influencers hierarchy improved: 56.4
csr communities hierarchy improved: 51.8
csr communities shapley: 50.699999999999996


SIR model:
influencers hierarchy with centrality: 473.09999999999997
influencers kkt: 472.4
influencers hierarchy improved: 471.40000000000003
influencers hierarchy orig: 4

In [ ]:
for 